In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import os.path
import datetime 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from surveysim.wrapper import surveySim

In [13]:
def collect_tile_id(start, end):
    day_now = start
    id_list = np.empty((0), dtype=int)
    id_plan = np.empty((0), dtype=int)
    while(day_now <= end):
        day_now = day_now + datetime.timedelta(days=1)
        listdirName = "{}{}{:02d}{:02d}{}".format('obslist',day_now.year, day_now.month, day_now.day, '.fits')   
        plandirName = "{}{}{:02d}{:02d}{}".format('obsplan',day_now.year, day_now.month, day_now.day, '.fits')    

        #print(day_now,listdirName)
        if os.path.exists(listdirName):
            a = Table.read(listdirName)
            set_a = set(a['TILEID'])
            set_id = set(id_list)
            set_diff = set_a - set_id
            id_list = np.append(id_list, np.array(list(set_diff), dtype=int)) #only add the IDs that don't repeat
        
        if os.path.exists(plandirName):
            a = Table.read(plandirName)
            set_a = set(a['TILEID'])
            set_id = set(id_plan)
            set_diff = set_a - set_id
            id_plan = np.append(id_plan, np.array(list(set_diff), dtype=int)) #only add the IDs that don't repeat
    
    return {'OBSERVED':id_list, 'PLANNED':id_plan}

In [14]:

init = datetime.date(2018, 4, 3)
fiberassign_cadence = datetime.timedelta(days=30)
obs_cadence = datetime.timedelta(days=1)
now = init
for i in range(5):
    now =  now + fiberassign_cadence
    print(now)

2018-05-03
2018-06-02
2018-07-02
2018-08-01
2018-08-31


In [26]:
start = datetime.date(2018, 2, 3)
n_epochs = 10
days_per_epoch = 30

fiberassign_cadence = datetime.timedelta(days=days_per_epoch)
end = start + fiberassign_cadence 

for i in range(n_epochs):
    surveySim(start.day, start.month, start.year, end.day, end.month, end.year)
    dd = collect_tile_id(start, end)
    print dd['OBSERVED'], len(dd['OBSERVED']), len(dd['PLANNED'])
    np.savetxt('gather_epoch{}.txt'.format(i), dd['OBSERVED'], fmt='%d')
    np.savetxt('epoch{}.txt'.format(i), dd['PLANNED'], fmt='%d')
    start = start + fiberassign_cadence + obs_cadence
    end = start + fiberassign_cadence

On the night starting  2018-02-04 01:06:00.000 , we observed  0  tiles.
On the night starting  2018-02-05 01:07:00.000 , we observed  1  tiles.
On the night starting  2018-02-06 01:08:00.000 , we observed  0  tiles.
On the night starting  2018-02-07 01:09:00.000 , we observed  0  tiles.
On the night starting  2018-02-08 01:10:00.000 , we observed  0  tiles.
On the night starting  2018-02-09 01:10:00.000 , we observed  0  tiles.
On the night starting  2018-02-10 01:11:00.000 , we observed  1  tiles.
On the night starting  2018-02-11 01:12:00.000 , we observed  5  tiles.
On the night starting  2018-02-12 01:13:00.000 , we observed  3  tiles.
On the night starting  2018-02-13 01:14:00.000 , we observed  5  tiles.
On the night starting  2018-02-14 01:15:00.000 , we observed  0  tiles.
On the night starting  2018-02-15 01:16:00.000 , we observed  9  tiles.
On the night starting  2018-02-16 01:17:00.000 , we observed  2  tiles.
On the night starting  2018-02-17 01:17:00.000 , we observed  0 

In [ ]:
len(id_plan), len(id_list)
id_list

In [ ]:
len(id_list), len(id_plan)
id_list